In [42]:
import os
import numpy as np
import pandas as pd
import nltk
import re
import copy
from nltk import Tree
from nltk.stem import WordNetLemmatizer

In [107]:
patterns = """
    NP:    {<DT><WP><VBP>*<RB>*<VBN><IN><NN>}
           {<NN|NNS|NNP|NNPS><IN    >*<NN|NNS|NNP|NNPS>+}
           {<JJ>*<NN|NNS|NNP|NNPS><CC>*<NN|NNS|NNP|NNPS>+}
           {<JJ>*<NN|NNS|NNP|NNPS>+}
    """
select_type = ['NN', 'NNS', 'VB', 'VBD', 'VBP', 'VBZ']
typer = nltk.RegexpParser(patterns)
letter_filter = re.compile('[^a-zA-Z]')
lemmatizer = WordNetLemmatizer()

In [108]:
training_descrption_files = [os.path.join('data/descriptions_train', filename) \
                             for filename in os.listdir('data/descriptions_train') 
                             if filename.endswith('.txt')]
training_tag_files = [os.path.join('data/tags_train', filename)\
                      for filename in os.listdir('data/tags_train')\
                      if filename.endswith('.txt')]
test_descrption_files = [os.path.join('data/descriptions_test', filename) \
                             for filename in os.listdir('data/descriptions_test') 
                             if filename.endswith('.txt')]
training_descrption_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
training_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
test_descrption_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))

In [5]:
def remove_empty(a):
    return [i for i in a if len(i) > 0]

In [6]:
def find_type(s, type_select):
    dictionary = set()
    sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(s)]
    sentences = [[letter_filter.sub('', sentences[i][j].lower())\
              for j in range(len(sentences[i]))] for i in range(len(sentences))]
    sentences = [remove_empty(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    for sent in sentences:
        for e in sent:
            if e[1] in type_select:
                dictionary.add(lemmatizer.lemmatize(e[0]))
    return dictionary

In [43]:
def define_des_tag_vector():
    selected = {}
    for file in training_descrption_files:
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            nvj = find_type(sent, select_type)
            for word in nvj:
                if word in selected.keys():
                    selected[word] += 1
                else:
                    selected[word] = 1
    for file in training_tag_files:
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            validTag = letter_filter.sub('', sent.split(':')[1])
            selected[validTag] = 100
    
    final = set()
    for k in selected.keys():
        if selected[k] >= 15 and selected[k] <= 2000:
            final.add(k)
    
    final_copy = copy.deepcopy(final)
    for element in final:
        if element in tags_final:
            final_copy.remove(element)
    
    return final_copy

In [44]:
a = define_des_tag_vector()

In [38]:
tags_final = []
for tag in tagList:
    if not tag in a:
        tags_final = tags_final + tag.split()
    else:
        tags_final.append(tag)
len(tags_final)


95

In [65]:
allTagFiles = [filename for filename in os.listdir('data/tags_train') if filename.endswith('.txt')]
tags = set()
for tagFile in allTagFiles:
    f = open(os.path.join('data/tags_train',tagFile), 'r+')
    for line in f:
        tag = line.split(':')[1][:-1] if line.split(':')[1].endswith('\n') else line.split(':')[1]
        tags.add(tag)
tagList = [tag for tag in tags]

In [84]:
train_tag_files = [os.path.join('data/tags_train', filename) \
                             for filename in os.listdir('data/tags_train') 
                             if filename.endswith('.txt')]
test_tag_files = [os.path.join('data/tags_test', filename) \
                             for filename in os.listdir('data/tags_test') 
                             if filename.endswith('.txt')]
train_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
test_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
test_tag_files[:1]

['data/tags_test/0.txt']

In [99]:
Y_training_1 = []
for file in training_descrption_files:
    vector_1 = dict.fromkeys(a, 0)
    sentences = [line for line in open(file, 'r+')]
    for sent in sentences:
        for word in sent.split():
            i = letter_filter.sub('', word.lower())
            if i in a:
                vector_1[i] += 1
    
    Y_training_1.append(list(vector_1.values()))

Y_training_2 = []
for tagFile in train_tag_files:
    vector_2 = dict.fromkeys(tagList, 0)
    f = open(os.path.join(tagFile), 'r+')
    for line in f:
        tag = line.split(':')[1][:-1] if line.split(':')[1].endswith('\n') else line.split(':')[1]
        if tag in tagList:
            vector_2[tag] += 1
    Y_training_2.append(list(vector_2.values()))

Y_training = []
for i in range(10000):
    combine = Y_training_1[i] + Y_training_2[i]
    Y_training.append(combine)

In [100]:
Y_test_1 = []
for file in test_descrption_files:
    vector_1 = dict.fromkeys(a, 0)
    sentences = [line for line in open(file, 'r+')]
    for sent in sentences:
        for word in sent.split():
            i = letter_filter.sub('', word.lower())
            if i in a:
                vector_1[i] += 1
    
    Y_test_1.append(list(vector_1.values()))

Y_test_2 = []
for tagFile in test_tag_files:
    vector_2 = dict.fromkeys(tagList, 0)
    f = open(os.path.join(tagFile), 'r+')
    for line in f:
        tag = line.split(':')[1][:-1] if line.split(':')[1].endswith('\n') else line.split(':')[1]
        if tag in tagList:
            vector_2[tag] += 1
    Y_test_2.append(list(vector_2.values()))


Y_test = []
for i in range(2000):
    combine = Y_test_1[i] + Y_test_2[i]
    Y_test.append(combine)

In [101]:
import pickle

pickle.dump( Y_training, open( "y_train_v2.p", "wb" ) )
pickle.dump( Y_test, open( "y_test_v2.p", "wb" ) )

In [106]:
find_type('she is dancing', select_type)

{'is'}